In [40]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
%load_ext autoreload

%autoreload 2
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
       # print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

SyntaxError: unexpected EOF while parsing (<ipython-input-40-5a2510698957>, line 18)

In [41]:
conv_base = tf.keras.applications.ResNet50(
    include_top=False,
    weights='imagenet')


In [42]:
dir =  "/kaggle/input/data"
train = os.listdir("/kaggle/input/data")

In [43]:
image1 =  os.path.join(dir + "images_001")

In [44]:
from glob import glob 
import pandas as pd

dataset = pd.read_csv('../input/data/Data_Entry_2017.csv')
paths = {os.path.basename(x): x for x in 
                   glob(os.path.join('..', 'input', 'data',  'images*', '*', '*.png'))}
print('Scans found:', len(paths), ', Total Headers', dataset.shape[0])
dataset['path'] = dataset['Image Index'].map(paths.get)
dataset['Cardiomegaly'] = dataset['Finding Labels'].map(lambda x: 'Cardiomegaly' in x)
dataset['Finding'] = dataset['Finding Labels'].map(lambda x: x != 'No Finding').astype(int)
dataset['Patient Age'] = np.clip(dataset['Patient Age'], 5, 100)
label_freq = dataset['Finding Labels'].apply(lambda s: str(s).split('|')).explode().value_counts().sort_values(ascending=False)
rare = list(label_freq[label_freq<10].index)
dataset['labels'] = dataset['Finding Labels'].apply(lambda s: [l for l in str(s).split('|') if l not in rare])



Scans found: 112120 , Total Headers 112120


In [45]:
#dataset['Finding Labels'] = dataset['Finding Labels'].map(lambda x: x.replace('No Finding', 'n'))
from itertools import chain
all_labels = np.unique(list(chain(*dataset['Finding Labels'].map(lambda x: x.split('|')).tolist())))
all_labels = [x for x in all_labels if len(x)>0]
print('All Labels ({}): {}'.format(len(all_labels), all_labels))
for c_label in all_labels:
    if len(c_label)>1: # leave out empty labels
        dataset[c_label] = dataset['Finding Labels'].map(lambda finding: 1.0 if c_label in finding else 0)
#dataset.sample(3)
dataset['disease_vec'] = dataset.apply(lambda x: [x[all_labels].values], 1).map(lambda x: x[0])

All Labels (15): ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']


In [46]:
data = dataset[['Image Index' , 'Patient ID', 'path', 'Finding' , 'labels' , 'disease_vec']]

In [47]:
noFind = data[data['Finding']==0]['Image Index'].tolist()
Find = data[data['Finding']==1]['Image Index'].tolist()
noFind_path = data[data['Finding']==0]['path'].tolist()
Find_path = data[data['Finding']==1]['path'].tolist()

In [48]:
path = (glob(os.path.join('..' , 'input' , 'data' , 'images*' , '*' , '*' )))

In [49]:
# #noFind_paths = []
# #for i in noFind[:10000]:
#     destination = (glob(os.path.join('..' , 'input' , 'data' , 'images*' , '*' , i )))
#     noFind_paths.append(destination)

In [50]:
# Find_paths = []
# for i in Find:
#     destination = (glob(os.path.join('..' , 'input' , 'data' , 'images*' , '*' , i )))
#     Find_paths.append(destination)

In [51]:
final_paths = Find_path + noFind_path[:10000]

final_data = data[data['Finding']==1]
temp= data[data['Finding']==0][:10000]


final_data.append(temp)
final_paths = final_data['path'].tolist()

In [52]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
train_df, valid_df , train_y , valid_y = train_test_split(final_paths, 
                                                          final_data['disease_vec'].tolist(),
                                   test_size = 0.6, 
                                   random_state = 2000,
                                   #stratify = data['labels'].map(lambda x: x[:4]
                                     )

valid_df, test_df , valid_y , test_y = train_test_split(valid_df, 
                                                        valid_y,
                                   test_size = 0.5, 
                                   random_state = 2000)



def flow_from_dataframe(img_data_gen, in_df, path_col, y_col, **dflow_args):
    base_dir = os.path.dirname(in_df[path_col].values[0])
    print('## Ignore next message from keras, values are replaced anyways')
    df_gen = img_data_gen.flow_from_directory(base_dir, 
                                     class_mode = 'sparse',
                                    **dflow_args)
    df_gen.filenames = in_df[path_col].values
    df_gen.classes = np.stack(in_df[y_col].values)
    df_gen.samples = in_df.shape[0]
    df_gen.n = in_df.shape[0]
    df_gen._set_index_array()
    df_gen.directory = '' # since we have the full path
    print('Reinserting dataframe: {} images'.format(in_df.shape[0]))
    return df_gen





In [53]:
SHUFFLE_BUFFER_SIZE = 1024
BATCH_SIZE = 32 
AUTOTUNE = tf.data.experimental.AUTOTUNE 
CHANNELS = 3
IMG_SIZE = 128

def parse_function(filename, label):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=CHANNELS)
    image_resized = tf.image.resize(image_decoded, [IMG_SIZE, IMG_SIZE])
    image_normalized = image_resized / 255.0
    return image_normalized, label



In [54]:


def create_dataset(filenames, labels, is_training=True):

    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    dataset = dataset.map(parse_function, num_parallel_calls=AUTOTUNE)
    
    if is_training == True:
        dataset = dataset.cache()
        dataset = dataset.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE)
        
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    
    return dataset



In [55]:
LR = 0.001
EPOCHS = 1

In [56]:
#train_df

IMG_SIZE = (128,128)

datagen_train = ImageDataGenerator(rescale = 1./255,
                                featurewise_center=True,
                                featurewise_std_normalization=True,
                                rotation_range=20,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                horizontal_flip=False,
                                shear_range=0.2,
                                zoom_range=0.2,
                                fill_mode = 'nearest')

datagen_valid = ImageDataGenerator(rescale = 1./255)
datagen_test = ImageDataGenerator(rescale = 1./255)





train_gen = flow_from_dataframe(datagen_train,train_df , 
                             path_col = 'path',
                            y_col = 'disease_vec', 
                            target_size = IMG_SIZE,
                             color_mode = 'rgb',
                            batch_size = 32)

valid_gen = flow_from_dataframe(datagen_valid, valid_df, 
                             path_col = 'path',
                            y_col = 'disease_vec', 
                            target_size = IMG_SIZE,
                             color_mode = 'rgb',
                            batch_size = 256) 

valid_gen = flow_from_dataframe(datagen_test, test_df, 
                             path_col = 'path',
                            y_col = 'disease_vec', 
                            target_size = IMG_SIZE,
                             color_mode = 'rgb',
                            batch_size = 256) 

# test_X, test_Y = next(flow_from_dataframe(core_idg, 
#                                valid_df, 
#                              path_col = 'path',
#                             y_col = 'disease_vec', 
#                             target_size = IMG_SIZE,
#                              color_mode = 'rgb',
#                             batch_size = 1024)) # one big batch


In [57]:
import tensorflow as tf
model = tf.keras.models.Sequential([
                                               tf.keras.layers.Conv2D(16 , (3,3) , activation='relu', input_shape =(128, 128 ,3)),
                                               tf.keras.layers.MaxPool2D(2,2),
                                               tf.keras.layers.Conv2D(64 , (3,3) , activation='relu'),
                                               tf.keras.layers.MaxPool2D(2,2),
                                               tf.keras.layers.Conv2D(256 , (3,3) , activation='relu'),
                                               tf.keras.layers.MaxPool2D(2,2),
                                                tf.keras.layers.Conv2D(512 , (3,3) , activation='relu'),
                                               tf.keras.layers.MaxPool2D(2,2),  
                                               tf.keras.layers.Flatten(),
                                               tf.keras.layers.Dense(128 , activation = 'relu'),
                                                tf.keras.layers.Dropout(0.25),
                                               tf.keras.layers.Dense(32 , activation = 'relu'),
                                                tf.keras.layers.Dropout(0.25),
                                                tf.keras.layers.Dense(16 , activation = 'relu'),
                                               tf.keras.layers.Dense(15 , activation = 'sigmoid')
                                               ])

In [58]:
train_ds = create_dataset(train_df, train_y)
val_ds = create_dataset(valid_df, valid_y)
test_ds = create_dataset(test_df, test_df)


In [59]:
@tf.function
def macro_soft_f1(y, y_hat):
    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    cost = 1 - soft_f1
    macro_cost = tf.reduce_mean(cost)
    return macro_cost


In [60]:
@tf.function
def macro_f1(y, y_hat, thresh=0.5):
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [61]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
  loss=macro_soft_f1,
  metrics=[macro_f1])

In [62]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mping
# img = mping.imread(final_paths[0])
# plt.imshow(img)

In [63]:
history = model.fit(train_ds,
                steps_per_epoch = 155,
              epochs = 200,
              validation_steps = 93,
                    validation_data=create_dataset(valid_df , valid_y))

Train for 647 steps, validate for 486 steps
Epoch 1/20
647/647 [==============================] - 394s 609ms/step - loss: 0.8344 - macro_f1: 0.1652 - val_loss: 0.8375 - val_macro_f1: 0.1625
Epoch 2/20
647/647 [==============================] - 14s 22ms/step - loss: 0.8343 - macro_f1: 0.1658 - val_loss: 0.8806 - val_macro_f1: 0.1194
Epoch 3/20
647/647 [==============================] - 14s 22ms/step - loss: 0.8463 - macro_f1: 0.1537 - val_loss: 0.8504 - val_macro_f1: 0.1496
Epoch 4/20
647/647 [==============================] - 14s 22ms/step - loss: 0.8613 - macro_f1: 0.1387 - val_loss: 0.9109 - val_macro_f1: 0.0891
Epoch 5/20
647/647 [==============================] - 15s 23ms/step - loss: 0.8781 - macro_f1: 0.1219 - val_loss: 0.9461 - val_macro_f1: 0.0539
Epoch 6/20
647/647 [==============================] - 14s 22ms/step - loss: 0.9001 - macro_f1: 0.0999 - val_loss: 0.9397 - val_macro_f1: 0.0603
Epoch 7/20
647/647 [==============================] - 14s 21ms/step - loss: 0.8893 - macro

KeyboardInterrupt: 

In [ ]:
model.predict(test_ds[:10])

In [ ]:
model.summary()

In [67]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer = RMSprop(lr= 0.001) ,
             loss=macro_soft_f1,
            metrics=[macro_f1 , 'acc'])

In [ ]:
#print(datagen_valid)

history = model.fit_generator(datagen_train,
                              validation_data=datagen_valid,
                              steps_per_epoch=200,
                              epochs=10,
                              validation_steps=10,
                              verbose=2)